In [66]:
import json
import os

import numpy as np
import pandas as pd

# Get the interactive Tools for Matplotlib
#%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#from sklearn.decomposition import PCA

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from tokenize_uk import tokenize_words

### Download vectors of words

https://web.stanford.edu/class/cs224n/materials/Gensim%20word%20vector%20visualization.html

In [6]:
glove_vec_path = '/home/nata/PycharmProjects/UCU/NLP/Module4/fiction.cased.tokenized.glove.300d'
vec_path = '/home/nata/PycharmProjects/UCU/NLP/Module4/fiction.cased.tokenized.glove_vec.300d'

In [7]:
glove_file = datapath(glove_vec_path)
word2vec_glove_file = get_tmpfile(vec_path)
glove2word2vec(glove_file, word2vec_glove_file)

(116804, 300)

In [8]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [24]:
len(model["постачання"])

300

In [47]:
print(model.doesnt_match("Львів борода Київ Одеса".split())) 

борода


/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [31]:
model.most_similar('Карпати')

[('Велетенські', 0.5601865649223328),
 ('гори', 0.5524407625198364),
 ('Альпи', 0.536340594291687),
 ('Собакаревої', 0.5013687610626221),
 ('несходимі', 0.4807429611682892),
 ('Кримські', 0.48002350330352783),
 ('крейдяні', 0.471161812543869),
 ('Холодної', 0.4687192440032959),
 ('лисі', 0.46492454409599304),
 ('Зазеленіли', 0.4571613073348999)]

In [34]:
result = model.most_similar(positive=['жінка', 'князь'], negative=['чоловік'])
print("{}: {:.4f}".format(*result[0]))
result = model.most_similar(positive=['чоловік', 'княгиня'], negative=['жінка'])
print("{}: {:.4f}".format(*result[0]))
result = model.most_similar(positive=['чоловік', 'королева'], negative=['жінка'])
print("{}: {:.4f}".format(*result[0]))

княгиня: 0.5063
князь: 0.4479
покійник: 0.3952


### Import corpus

In [37]:
texts = []
targets = []
for file in os.listdir("1551/json"):
    with open("1551/json/" + file) as f:
        j = json.load(f)
        texts.append(j[0]["CallZText"])
        targets.append(j[0]["CallZType"])

In [73]:
targets = pd.Series(targets)
texts = pd.Series(texts)
len(texts),len(targets)

(127329, 127329)

In [74]:
top20 = targets.value_counts()[:20].index
texts_top20 = texts[targets.isin(top20)]
targets_top20 = targets[targets.isin(top20)]
len(texts_top20),len(targets_top20)

(61537, 61537)

In [86]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(texts_top20, targets_top20, test_size = 0.2, random_state = 42)

In [94]:
y_train.nunique(), y_test.nunique()

(20, 20)

### Vectorization

In [87]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer = tokenize_words)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [88]:
X_train_transformed.shape

(49229, 134852)

In [89]:
len(vectorizer.get_feature_names())

134852

### Classification

In [91]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr = lr.fit(X_train_transformed, y_train)

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [93]:
from sklearn.metrics import classification_report

test_preds = lr.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.73      0.68      0.71       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.75      0.66      0.70       485
                                                                    Відсутнє ХВП       0.86      0.80      0.83       305
                                                                 Відсутність ГВП       0.89      0.91      0.90      2659
                                                            Відсутність опалення       0.80      0.85      0.83      1241
                                                  Відсутність опалення по стояку       0.72      0.65      0.68       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.68      0.76      0.72       544
                       